# Data Exploration - Stellenanzeigen

## Data Preperation

### Import

In [ ]:
# Download and import of German Stopwords, execution only needed once
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

from matplotlib.colors import LogNorm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from langdetect import detect, DetectorFactory
from ast import literal_eval
from fuzzywuzzy import fuzz

In [ ]:
# Loading synonyms into a dataframe
kode = pd.read_excel("kode_data.xlsx", header=1)

# Keep one of the df commented:
columns = [ 'title', 
            'location', 
            'company',
            'job_dt',
            'year_month',
            'contract_type',
            'work_type',
            'pers_resp',
            'language',
            'introduction_clean_tokens', 
            'description_clean_tokens',
            'profile_clean_tokens', 
            'weoffer_clean_tokens']


# Reading full dataset into dataframe
df = pd.read_csv('jobs_tokenized.csv', encoding='UTF-8', usecols=columns)

# Reading sample dataset into dataframe
# df = pd.read_excel("jobs_sample.xlsx")